
Please __read the document carefuly__ and submit your code accordingly.  

Once you fill in the functions as explained below, disconnect runtime, reconnect and run all (i.e. ```Runtime -> Run all```) in order to double check that it still works. Delete any test code you have so that I do just see the functions below, I will write my own test code, clear all outuput (i.e. ```Edit -> Clear all outputs```), finally save this file as "**W4_studentID.ipynb**" and submit via [ODTU class](https://odtuclass2024f.metu.edu.tr/mod/assign/view.php?id=68205).  

Use of AI tools (such as the built-in Gemini in colab, or anyother you like) is allowed. However, if you use an AI tool, add the prompt(s) you used as a comment to the beginning of each code cell.  
Along with the comment, explain if the first prompt worked, if not explain how you fixed it, add all versions of your prompts in to your comments.


Allowed imports:  
- ```numpy```.  

Any submission:  
- with test code,  
- that crashes,  
- any other import than mentioned above
- not properly named

will not be graded.


## Full name: Çağdaş Güven

## Student ID 2738938

### Generation of subspaces with desired guaranteed minimal or maximal angles

1. **Type and value validation**: The function first checks that the input parameters are of the correct types (integer, float) and have valid values ( positive, within the range of the ambient space).

2. **Geometric possibility check**: The function checks if it's geometrically possible to generate the two subspaces given the provided parameters. If the sum of the subspace dimensions is greater than the ambient space dimension and the minimum angle is greater than 0, the function returns two empty matrices.

3. **Generate basis for first subspace**: The function generates a random basis `B1` for the first subspace `S1` and then orthonormalizes it using QR decomposition. 


An example for A Matrix using QR decomposition
$$ 
A = \begin{bmatrix} 1 & 2 \\
                    3 & 4 \\
                    5 & 6
    \end{bmatrix}
$$

$A = QR $

Q will be 3X2 matrix with orthonormal colums.

R will be 2x2 upper triangular matrix.

Using the Gram-Schmidt Process our vectors will be 

$$ 
a_1 = \begin{bmatrix}   1  \\
                        3  \\
                        5 
        \end{bmatrix}

a_2 = \begin{bmatrix}    2 \\
                         4 \\
                         6
        \end{bmatrix}
$$

calculating $q_1 = a_1||a_1||$ with $l_2$ norm

$||a_1|| = \sqrt{1^2 + 3^2 + 5^2}= \sqrt{35}$ so $q_1 = \frac{1}{||a_1||} a_1 $

then we project $a_2$ onto $q_1 = (a_2 . q_1)q_1$ and substract this from $a_2$ to make $q_2$ orthogonal to $q_1$

$$
a_2 . q_1 = \begin{bmatrix}  2 & 4 & 6
                \end{bmatrix}

                .
                \begin{bmatrix} \frac{1}{\sqrt{35}} \\
                                \frac{3}{\sqrt{35}} \\
                                \frac{5}{\sqrt{35}} 
                \end{bmatrix}

                = \frac{2}{\sqrt{35}} + \frac{12}{\sqrt{35}} + \frac{30}{\sqrt{35}} = \frac{44}{\sqrt{35}}    
$$
then 
$ q_2 = a_2 - (\frac{4}{\sqrt{35}} q_1) $

After normalizing $q_2$, we get the columns of Q

then R matrix is calculated using the dot products of A's columns with the columns of Q, which gives us an upper triangular matrix.

4. **Generate basis for second subspace**: The function generates a random basis `B2` for the second subspace `S2` and then orthonormalizes it. However, it needs to ensure that the minimum angle between the two subspaces is at least the provided `minimalAngle`. To do this, it performs a loop with a maximum of 1000 attempts, generating a new `B2` and checking the minimal angle until a valid one is found or the maximum attempts are exceeded.

5. **Generate random coefficients**: The function generates random coefficient matrices `C1` and `C2` for the data points in each subspace.

6. **Generate data matrices**: The function computes the data matrices `M1` and `M2` by multiplying the basis matrices `B1` and `B2` with the corresponding coefficient matrices `C1` and `C2`.

7. **Error handling**: The function includes a try-except block to handle any exceptions that may occur during the execution of the code.

The function returns the two data matrices `M1` and `M2` if the generation was successful, or two empty numpy arrays if any errors occurred or the input parameters were invalid.


So for GenerateSubspacesMin I have requested Claude of its support 

I have used comment block as first input 

then I got some crude code which correctly generated data for me

added logic checks and uploaded again

Asked it to remove np.linalg.svd calculations

In [ ]:
import numpy as np

def GenerateSubspacesMin(D=5, dS1=2, dS2=2, nS1=50, nS2=50, minimalAngle=0):
    '''
    This function generates 2 data matrices in subspaces S1 and S2 respectively that live in D dimensional ambient space
    dSi is the dimension of subspace Si
    nSi is the number of data points to be generated in subspace Si
    minimalAngle is the minimal angle between subspaces, recall the definition in lecture notes
    Function returns two numpy arrays: M1, M2
    where:
        dimension of matrix Mi is (D, nSi)
        rank(Mi) = dSi
    if passed data does not make sense, return two empty numpy arrays
    '''
    try:
        # Type checking
        if not all(isinstance(x, (int, np.integer)) for x in [D, dS1, dS2, nS1, nS2]):
            return np.array([]), np.array([])
        if not isinstance(minimalAngle, (int, float, np.integer, np.floating)):
            return np.array([]), np.array([])
            
        # Value validation
        if not (0 <= minimalAngle <= np.pi/2):
            return np.array([]), np.array([])
        if not all(x > 0 for x in [D, dS1, dS2, nS1, nS2]):
            return np.array([]), np.array([])
        if not (dS1 <= D and dS2 <= D):
            return np.array([]), np.array([])
            
        # Check geometric possibility
        if D < dS1 + dS2 and minimalAngle > 0:
            return np.array([]), np.array([])

        # Generate random basis for first subspace
        B1 = np.random.randn(D, dS1)
        # Orthonormalize using QR decomposition
        Q1, R1 = np.linalg.qr(B1)
        B1 = Q1[:, :dS1]

        # Generate random basis for second subspace
        max_attempts = 1000
        attempt = 0
        success = False
        
        while attempt < max_attempts:
            try:
                B2 = np.random.randn(D, dS2)
                Q2, R2 = np.linalg.qr(B2)
                B2 = Q2[:, :dS2]
                
                # Compute the minimal angle between subspaces
                # using the maximum absolute inner product of basis vectors
                max_inner_product = np.max(np.abs(B1.T @ B2))
                min_angle = np.arccos(max_inner_product)
                
                if min_angle >= minimalAngle:
                    success = True
                    break
                    
            except np.linalg.LinAlgError:
                pass
                
            attempt += 1
        
        if not success:
            return np.array([]), np.array([])

        # Generate random coefficients for points in each subspace
        C1 = np.random.randn(dS1, nS1)
        C2 = np.random.randn(dS2, nS2)

        # Generate points in each subspace
        M1 = B1 @ C1  # D x nS1 matrix
        M2 = B2 @ C2  # D x nS2 matrix

        return M1, M2
        
    except Exception:
        return np.array([]), np.array([])




### Finding minimal and maximal angles between subspaces  

1. The function takes two input matrices, `M1` and `M2`, which represent data matrices containing data points that span subspaces `S1` and `S2`, respectively.

2. The function calculates and returns the minimal angle between the subspaces `S1` and `S2`. The minimal angle is defined as the smallest principal angle between the subspaces spanned by the columns of `M1` and `M2`.

3. The function first performs input validation to ensure that the input matrices are valid:
   - Checks if `M1` and `M2` are numpy arrays
   - Checks if the input matrices are empty
   - Checks if the matrices have the same number of rows (i.e., the same ambient dimension)
   - Checks if the matrices contain only numeric values (no NaN or Inf)

4. If any of the input validation checks fail, the function returns `-1000` to indicate an error.

5. If the input is valid, the function proceeds to find the linearly independent columns of `M1` and `M2` using Gaussian elimination. This is done by iterating through the columns of each matrix and keeping only the columns that are not all-zero or all-equal.

6. The function then computes the minimal angle between the subspaces `S1` and `S2` by finding the maximum absolute inner product of the basis vectors of the two subspaces. The minimal angle is calculated as the arccos of this maximum absolute inner product.

7. Finally, the function returns the minimal angle in radians.

In summary, this function calculates the minimal angle between two subspaces represented by the input data matrices `M1` and `M2`. It first performs input validation, then finds the linearly independent basis vectors of the subspaces, and finally computes the minimal angle using the maximum absolute inner product of the basis vectors.


this time it was tricky after completing similar methods I got a function that utilizes svd decomposition

after changing np.linalg.svd to np linalg.matrix_rank I have noticed it is still utilizing svd calculations

so there is a flag for Hermitian. But with that modification max from 50x50 matrix for nSi returns angle value close to 0 

From what I understand from that pool of data created from 'max_inner_product = np.max(np.abs(Q1.T @ Q2))
'
of 2 subspaces there would always vectors closely related to each other 

I have tested eliminating top %20 data and only getting average value for testing but np.linalg.matrix_rank( , hermitian=True) would always find insensible value. 

so I just uploaded the code and asked for it to calculate it after gaussian elimination

```
Q1, R1 = np.linalg.qr(M1)
            # Keep only the linearly independent columns
            Q1 = Q1[:, :np.linalg.matrix_rank(M1, hermitian=True)]
            
            Q2, R2 = np.linalg.qr(M2)
            Q2 = Q2[:, :np.linalg.matrix_rank(M2, hermitian=True)]
            
            # Compute the minimal angle between subspaces
            # using the maximum absolute inner product of basis vectors
            max_inner_product = np.max(np.abs(Q1.T @ Q2))
            min_angle = np.arccos(max_inner_product)
            
            return float(min_angle)
```
above is the older QR decomposition approach similar in the generation function

In [ ]:

def FindMinimalAngles(M1, M2):
    '''
    This function calculates and returns the minimal angle between subspaces S1, and S2
    that contain the data points M1 and M2 respectively.
    The minimal angle between subspaces is defined as the smallest principal angle
    between the subspaces spanned by the columns of M1 and M2.
    
    Parameters:
    M1, M2: numpy arrays representing data matrices
    
    Returns:
    float: minimal angle in radians between the subspaces
    -1000: if input data is invalid
    '''
    try:
        # Input validation
        if not isinstance(M1, np.ndarray) or not isinstance(M2, np.ndarray):
            return -1000
        
        # Check if matrices are empty
        if M1.size == 0 or M2.size == 0:
            return -1000
        
        # Check if matrices have same number of rows (same ambient dimension)
        if M1.shape[0] != M2.shape[0]:
            return -1000
        
        # Check if matrices are numeric
        if not (np.issubdtype(M1.dtype, np.number) and np.issubdtype(M2.dtype, np.number)):
            return -1000
            
        # Check for NaN or Inf values
        if np.any(np.isnan(M1)) or np.any(np.isnan(M2)) or \
           np.any(np.isinf(M1)) or np.any(np.isinf(M2)):
            return -1000
        
         # Find linearly independent columns for M1 using Gaussian elimination
        Q1 = np.zeros_like(M1)
        rank = 0
        for i in range(M1.shape[1]):
            col = M1[:, i]
            if np.allclose(col, np.zeros_like(col)) or np.all(np.isclose(col, col[0])):
                continue
            Q1[:, rank] = col / np.linalg.norm(col)
            rank += 1
        Q1 = Q1[:, :rank]
        
        # Find linearly independent columns for M2 using Gaussian elimination
        Q2 = np.zeros_like(M2)
        rank = 0
        for i in range(M2.shape[1]):
            col = M2[:, i]
            if np.allclose(col, np.zeros_like(col)) or np.all(np.isclose(col, col[0])):
                continue
            Q2[:, rank] = col / np.linalg.norm(col)
            rank += 1
        Q2 = Q2[:, :rank]
        

        # Compute the minimal angle between subspaces
        # using the maximum absolute inner product of basis vectors
        max_inner_product = np.max(np.abs(Q1.T @ Q2))
        min_angle = np.arccos(max_inner_product)
            
        return float(min_angle)
            
    except Exception:
        return -1000